<a href="https://colab.research.google.com/github/Yasmine-ChemP/GMCtraining/blob/main/NLP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Phase 1 : get the dataset
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt

(train_ds,val_ds,test_ds),info = tfds.load('imdb_reviews',
                       split=['train[:75%]','train[75%:100%]','test'],
                       shuffle_files=True,
                       with_info=True,
                       as_supervised=True)


In [ ]:
train_ds= train_ds.batch(32)
val_ds = val_ds.batch(32)
test_ds = test_ds.batch(32)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
max_length=100
training_samples=200
validation_samples=1000
max_words=20000
embedding_dim=300

# Let's make a text-only dataset (no labels):

vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_words,
 output_mode='int',
 output_sequence_length=max_length)

In [ ]:
text_ds= train_ds.map(lambda x, y: x)

In [ ]:
vectorize_layer.adapt(text_ds) #equivalent de tokenizer.fit

In [ ]:
vectorize_layer.get_config()

{'batch_input_shape': (None,),
 'dtype': 'string',
 'idf_weights': None,
 'max_tokens': 20000,
 'name': 'text_vectorization',
 'ngrams': None,
 'output_mode': 'int',
 'output_sequence_length': 100,
 'pad_to_max_tokens': False,
 'ragged': False,
 'sparse': False,
 'split': 'whitespace',
 'standardize': 'lower_and_strip_punctuation',
 'trainable': True,
 'vocabulary': None}

In [ ]:
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds) #Equivalent de tokenizer.fit
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), tf.reshape(label, (-1,1))

# Vectorize the data.
train = train_ds.map(vectorize_text)
val = val_ds.map(vectorize_text)
test = test_ds.map(vectorize_text)


#Do async prefetching /buffering of the data for the best performances 
train = train.cache().prefetch(buffer_size=10)
val= val.cache().prefetch(buffer_size=10)
test=test.cache().prefetch(buffer_size=10)

In [ ]:
for text, label in train.take(1):
  print(text[0])

tf.Tensor(
[   11    14    34   411   367    18    90    27  9790     8    33  1297
  4180    41   506 11462   193    25    86   152    19    11   215   316
    27    64   241   217     8   482    56    64    86   113    96    22
  5817    11    93   644   743    11    18     7    34   402 10038   168
  2450   412     2    88  1207   137    68   144    52     2     1  7184
    68   246    64  2845    16 18348  2805     1     1  1425  5138     3
    40     1  1630    17  4180    14   155    19     4  1207   856  7965
     8     4    18    12    14  3867     5    97   146  1219    10   231
   684    12    48    25], shape=(100,), dtype=int64)


In [ ]:
import tensorflow
#construction du modele
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(None,),dtype='int64')

#ajouter la couche d'embedding
x= layers.Embedding(20000,300)(inputs)
#x=layers.Dropout(0.5)(x)
#x=layers.Dense(128, activation='relu')(x)
#x=layers.SimpleRNN(32, dropout=0.3, recurrent_dropout=0.2, retun_sequence=True)(x)
#x=layers.SimpleRNN(32)
#x=layers.LSTM(32)(x)
x=layers.Bidirectional(layers.LSTM(32))(x)

#other model
#GRU
x=layers.GRU
output=layers.Dense(1,activation='sigmoid')(x)

model= tensorflow.keras.Model(inputs=inputs, outputs=output)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 300)         6000000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               85248     
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 6,085,313
Trainable params: 6,085,313
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train, epochs=1, validation_data=val )

586/586 [==============================] - 121s 200ms/step - loss: 0.4624 - accuracy: 0.7778 - val_loss: 0.3771 - val_accuracy: 0.8323


In [ ]:
model.evaluate(test)

782/782 [==============================] - 27s 34ms/step - loss: 0.4161 - accuracy: 0.8076


[0.4160822927951813, 0.8075600266456604]